FAILED EXPERIMENT -- forcing `Variable.register_hook` during `__init__` and `__new__` to try to register grad Variables with the worker when they're created during calls to `backward` downstream.

Could be possible to fix, and would result in cleaner code if we can, but not otherwise necessary at the prototyping stage.

### Notes

This requires using PyTorch v0.3.1.  Somewhere between 0.3.1 and 0.4.0 parts of the backend were significantly rewritten, preventing us from performing the following hacks. (Likely has to do with them fusing Variable and Tensor).  That may change once their new API stabilizes.

This work has led me to believe that we'll eventually just have one mode that will do everything we need (including keeping track of the model's version control tree for tracking contribution).

<a id='toc'></a>

# [Main event](#start)
[Wrappers](#wrap)<br>
[init and repr](#initrepr)<br>
[Helpers](#helps)<br>
[Hooking](#hook)<br>

# [Minimal tests for each Torch object](#tests)
[FloatTensor](#float_test)<br>
[DoubleTensor](#double_test)<br>
[HalfTensor](#half_test)<br>
[ByteTensor](#byte_test)<br>
[CharTensor](#char_test)<br>
[ShortTensor](#short_test)<br>
[IntTensor](#int_test)<br>
[LongTensor](#long_test)<br>
[Variable](#var_test) (TODO) <br>

# Imports

In [1]:
from grid.clients.torch import TorchClient

/home/jason/anaconda3/envs/openmined/lib/python3.6/site-packages/h5py-2.7.1-py3.6-linux-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
known_workers=['/p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX','/p2p-circuit/ipfs/QmXkWUybbTnfvFH8SUcrug6RGTLYTB23gSockKLxueR1vQ']
client = TorchClient(verbose = False, include_github_known_workers=False, known_workers = known_workers)


UPDATE: Connecting to IPFS... this can take a few seconds...

SUCCESS: Connected!!! - My ID: QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc

UPDATE: Querying known workers...
	WORKER: /p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/QmXkWUybbTnfvFH8SUcrug6RGTLYTB23gSockKLxueR1vQ...SUCCESS!!!

UPDATE: Searching for IPFS nodes - 27 found overall - 1 are OpenMined workers          

SUCCESS: Found 1 OpenMined nodes!!!



In [3]:
service_self = client.services['torch_service']

In [4]:
import torch
import inspect
from torch.autograd import Variable
import random
import re
from functools import wraps, partial, partialmethod
from types import *
from collections import OrderedDict
import imp
# from contextlib import contextmanager

In [5]:
tensorvar_types = [torch.FloatTensor,
                torch.DoubleTensor,
                torch.HalfTensor,
                torch.ByteTensor,
                torch.CharTensor,
                torch.ShortTensor,
                torch.IntTensor,
                torch.LongTensor,
                torch.autograd.variable.Variable]

<a id='start'></a>

# Main Event [(top)](#toc)

<a id='wrap'></a>

### Wrappers.
`assign_workers_function` is for functions (e.g. `torch.add(x ,y)`)<br>`assign_workers_method` is for methods (e.g. `x.add(y)`).

In [6]:
def assign_workers_function(worker_ids):
    def decorate(func):
        @wraps(func)
        def send_to_workers(*args, **kwargs):
            part = func(*args, **kwargs)
            command = compile_command(part)
            tensorvars = get_tensorvars(command)
            has_remote, multiple_owners = check_tensorvars(tensorvars)
            if not has_remote:
                result = part.func(*args, **kwargs)
                if type(result) in tensorvar_types:
                    result = service_self.register_object(result, False)
                return result
            elif multiple_owners:
                raise NotImplementedError('MPC not yet implemented: Torch objects need to be on the same machine in order to compute with them.')
            else:
                for worker in worker_ids:
                    print("Placeholder print for sending command to worker {}".format(worker))
                    args, kwargs = send_command(command)
                receive_commands(worker_ids)  ## Probably needs to happen async
                return args, kwargs
        return send_to_workers
    return decorate

In [7]:
def assign_workers_method(worker_ids):
    def decorate(method):
        @wraps(method)
        def send_to_workers(self, *args, **kwargs):
            part = method(self, *args, **kwargs)
            if self.is_pointer_to_remote:
                command = compile_command(part)
                for worker in worker_ids:
                    print("Placeholder print for sending command to worker {}".format(worker))
                    args, kwargs = send_command(command)
                receive_commands(worker_ids)  ## Probably needs to happen async
                return args, kwargs
            else:
                #import ipdb; ipdb.set_trace()
                result = part.func(self, *args, **kwargs)
                if type(result) == torch.autograd.variable.Variable:
                    result.data_registered = False
                    result.grad_registered = False
                if type(result) in tensorvar_types and not hasattr(result, 'owner'):
                    my_service = self.worker.services['torch_service']
                    result = my_service.register_object(result, False)
                return result
        return send_to_workers
    return decorate

Wrappers that do nothing except pass a copy of the underlying torch function/method, as well as any args/kwargs supplied.

In [8]:
def pass_func_args(func):
    @wraps(func)
    def pass_args(*args, **kwargs):
        return partial(func, *args, **kwargs)
    return pass_args

def pass_method_args(method):
    @wraps(method)
    def pass_args(*args, **kwargs):
        return partialmethod(method, *args, **kwargs)
    return pass_args

<a id='initrepr'></a>

### init, repr, new
Treat `__init__`, `__repr__`, and `__new__` specially in certain cases.  The `__init__` registers the object with the worker, the `__repr__` replaces torch's default pretty printing with a (less) pretty printing of our own, and `__new__` makes sure that new objects created in C as a result of computations are registered as well, even though their `__init__` is never called.

In [9]:
def hook_tensor___init__(service_self, tensor_type):
    def new___init__(self, *args):
        super(tensor_type, self).__init__()
        self = service_self.register_object(self,False)

    tensor_type.__init__ = new___init__
    
def hook_tensor___new__(service_self, tensor_type):
    tensor_type.old___new__ = tensor_type.__new__
    def new___new__(cls, *args, **kwargs):
        result = tensor_type.old___new__(cls, *args,  **kwargs)
        result = service_self.register_object(result, False)
        return result
    
    tensor_type.__new__ = new___new__

In [10]:
def hook_var_force_hook():
    def force_hook(self, hook):
        if self._backward_hooks is None:
            self._backward_hooks = OrderedDict()
            if self.grad_fn is not None:
                self.grad_fn._register_hook_dict(self)
        handle = torch.utils.hooks.RemovableHandle(self._backward_hooks)
        self._backward_hooks[handle.id] = hook
        return handle
    torch.autograd.variable.Variable.force_hook = force_hook

In [11]:
def set_grad_attr(grad, attr:str):
    new_grad = grad
    setattr(new_grad, attr, False)
    return new_grad

def hook_var___init__(service_self):
    def new___init__(self, *args, **kwargs):
        super(torch.autograd.variable.Variable, self).__init__(*args, **kwargs)
        self.data_registered =  False
        self.grad_registered =  False
        
        self.force_hook(lambda grad: set_grad_attr(grad, 'data_registered'))
        self.force_hook(lambda grad: set_grad_attr(grad, 'grad_registered'))
#         if not hasattr(self.data, 'owner'):
#             self.data = service_self.register_object(self.data,False)
#             self.data_registered = True
#         if self.grad is not None and not hasattr(self.grad, 'owner'):
#             self.grad = service_self.register_object(self.grad,False)
#             self.grad_registered = True

    torch.autograd.variable.Variable.__init__ = new___init__

**TODO**: Make sure that we're registering as little as possible.

In [12]:
def hook_var___new__(service_self):
    old_var___new__ = torch.autograd.variable.Variable.__new__
    def new___new__(cls, *args, **kwargs):
        result = old_var___new__(cls, *args,  **kwargs)
        result = service_self.register_object(result, False)
        result.data_registered = False
        result.grad_registered = False
        result.force_hook(lambda grad: set_grad_attr(grad, 'data_registered'))
        result.force_hook(lambda grad: set_grad_attr(grad, 'grad_registered'))
        return result
    
    torch.autograd.variable.Variable.__new__ = new___new__

In [13]:
def hook_var_contents(service_self):
    torch.autograd.variable.Variable.old_data = torch.autograd.variable.Variable.data
    torch.autograd.variable.Variable.old_grad = torch.autograd.variable.Variable.grad
    @property
    def new_data(self):
        if not self.data_registered:
            self.old_data = service_self.register_object(self.old_data, False)
            self.data_registered = True
        return self.old_data
    
    @property
    def new_grad(self):
        if not self.grad_registered:
            if self.old_grad is not None:
                self.old_grad = service_self.register_object(self.old_grad, False)
                self.grad_registered = True
        return self.old_grad
    
    torch.autograd.variable.Variable.data = new_data
    torch.autograd.variable.Variable.grad = new_grad

In [14]:
def hook_tensor___repr__(service_self, tensor_type):
        def __repr__(self):
            if(service_self.worker.id == self.owner):
                return self.old__repr__()
            else:
                return "[ {} - Location:{} ]".format(tensor_type, self.owner)

        # if haven't reserved the actual __repr__ function - reserve it now
        try:
            tensor_type.old__repr__
        except:
            tensor_type.old__repr__ = tensor_type.__repr__
            

        tensor_type.__repr__ = __repr__

<a id='helps'></a>

### Helpers.

In [15]:
def get_tensorvars(command):
    args = command['args']
    kwargs = command['kwargs']
    arg_types = command['arg_types']
    kwarg_types = command['kwarg_types']
    tensorvar_args = [args[i] for i in range(len(args)) if arg_types[i] in tensorvar_types]
    tensorvar_kwargs = [kwargs[i][1] for i in range(len(kwargs)) if kwarg_types[i] in tensorvar_types]
    return tensorvar_args + tensorvar_kwargs
    
def check_tensorvars(tensorvars):
    has_remote = any([tensorvar.is_pointer_to_remote for tensorvar in tensorvars])
    multiple_owners = len(set([tensorvar.owner for tensorvar in tensorvars])) != 1
    return has_remote, multiple_owners

These are unfinished.  They'll need to be integrated into the rest of Grid after this prototype is ready for merging.

In [16]:
def send_command(command):
    print(command['command'])
    print([type(arg) for arg in command['args']])
    print([type(pair) for pair in command['kwargs']])
    print('===========')
    print()
    return command['args'], command['kwargs']

def receive_commands(worker_ids):
    print('Placeholder print for receiving commands from workers in the following list')
    print(worker_ids)

In [17]:
def compile_command(partial_func):
    func = partial_func.func
    args = partial_func.args
    kwargs = partial_func.keywords
    command = {}
    command['command'] = func.__name__
    command['command_type'] = type(func)
    command['args'] = args
    command['kwargs'] = kwargs
    command['arg_types'] = [type(x) for x in args]
    command['kwarg_types'] = [type(kwargs[x]) for x in kwargs]
    return command

<a id='hook'></a>

# Where the hooking happens ([back to var](#var_test))
Timing is to see how much overhead we've added.  Nothing says this should scale linearly, but it's a quick benchmark.

In [18]:
%%time
for x in range(100000):
    y = torch.FloatTensor([[2,2],[2,2]])
    z = torch.FloatTensor([[1,1],[1,1]])
    res = y.add(z)

CPU times: user 1.26 s, sys: 36.6 ms, total: 1.3 s
Wall time: 1.28 s


In [19]:
%%time

for attr in dir(torch):
    if attr == 'typename':
        continue
    if type(torch.__getattribute__(attr)) in [FunctionType, BuiltinFunctionType]:
        torch.__setattr__(attr, assign_workers_function(['A1','B1', 'B2'])(pass_func_args(torch.__getattribute__(attr))))

# the first four can cause infinite recursion or internal type errors
exclude = ['ndimension', 'nelement', 'size','numel', 'ser', 'de']
var_exclude = ['__getattr__']

print("Hooking Variable")
hook_var_force_hook()
hook_var___init__(service_self)
hook_var___new__(service_self)
hook_var_contents(service_self)

for tensor_type in tensorvar_types:
    print('Hooking {}'.format(tensor_type))
    print('==============')
    if tensor_type is not torch.FloatTensor and tensor_type is not torch.autograd.variable.Variable:
        #hook_tensor___init__(service_self, tensor_type)
        hook_tensor___new__(service_self, tensor_type)
        hook_tensor___repr__(service_self, tensor_type)
    for attr in dir(tensor_type):
        if (tensor_type == torch.autograd.variable.Variable and attr in var_exclude) or attr in exclude:
            print(attr,' skipped')
            continue
        lit = getattr(tensor_type, attr)
        is_desc = inspect.ismethoddescriptor(lit)
        is_func = type(lit)==FunctionType
        is_mappingproxy = attr == '__dict__'
        try:
            is_service_func = 'TorchService' in lit.__qualname__
        except:
            is_service_func = False
        is_base = attr in dir(object)
        is_old = re.match('old*', attr) is not None
        if (is_desc or (is_func and not is_service_func)) and not is_base and not is_old:
            print(attr)
            setattr(tensor_type, 'old_{}'.format(attr), lit)
            setattr(tensor_type, attr, assign_workers_method(['A1','B1', 'B2'])(pass_method_args(lit)))
        else:
            print(attr, ' skipped')
    print()

Hooking Variable
Hooking <class 'torch.FloatTensor'>
__add__
__and__
__array__
__array_wrap__
__bool__
__class__  skipped
__deepcopy__
__delattr__  skipped
__delitem__
__dict__  skipped
__dir__  skipped
__div__
__doc__  skipped
__eq__  skipped
__float__
__format__  skipped
__ge__  skipped
__getattribute__  skipped
__getitem__
__getstate__
__gt__  skipped
__hash__  skipped
__iadd__
__iand__
__idiv__
__ilshift__
__imul__
__init__  skipped
__init_subclass__  skipped
__int__
__invert__
__ior__
__ipow__
__irshift__
__isub__
__iter__
__itruediv__
__ixor__
__le__  skipped
__len__
__long__
__lshift__
__lt__  skipped
__matmul__
__mod__
__module__  skipped
__mul__
__ne__  skipped
__neg__
__new__  skipped
__nonzero__
__or__
__pow__
__radd__
__rdiv__
__reduce__  skipped
__reduce_ex__  skipped
__repr__  skipped
__rmul__
__rpow__
__rshift__
__rsub__
__rtruediv__
__setattr__  skipped
__setitem__
__setstate__
__sizeof__  skipped
__str__  skipped
__sub__
__subclasshook__  skipped
__truediv__
__weakref_

abs
abs_
add
add_
addbmm
addbmm_
addcdiv
addcdiv_
addcmul
addcmul_
addmm
addmm_
addmv
addmv_
addr
addr_
apply_
baddbmm
baddbmm_
bernoulli_
bmm
byte
char
chunk
clamp
clamp_
clone
contiguous
copy_
cpu
cross
cuda
cumprod
cumsum
data  skipped
data_ptr
diag
dim
div
div_
dot
double
element_size
eq
eq_
equal
expand
expand_as
fill_
float
fmod
fmod_
gather
ge
ge_
geometric_
ger
gt
gt_
half
index
index_add_
index_copy_
index_fill_
index_select
int
is_contiguous
is_cuda  skipped
is_pinned
is_same_size
is_set_to
is_shared
is_signed
is_sparse  skipped
kthvalue
le
le_
long
lt
lt_
map2_
map_
masked_copy_
masked_fill_
masked_scatter_
masked_select
matmul
max
median
min
mm
mode
mul
mul_
mv
narrow
ndimension  skipped
ne
ne_
neg
neg_
nelement  skipped
new
nonzero
numel  skipped
numpy
old___new__  skipped
old__repr__  skipped
permute
pin_memory
prod
put_
random_
remainder
remainder_
repeat
resize_
resize_as_
scatter_
scatter_add_
select
set_
shape  skipped
share_memory_
short
sign
sign_
size  skipped
sort

In [20]:
%%time
for x in range(100000):
    y = torch.FloatTensor([[2,2],[2,2]])
    z = torch.FloatTensor([[1,1],[1,1]])
    res = y.add(z)

CPU times: user 2.16 s, sys: 527 ms, total: 2.69 s
Wall time: 2.2 s


<a id='tests'></a>

# Tests [(top)](#toc)

<a id='float_test'></a>

### FloatTensor

In [21]:
x = y.add(z)

In [22]:
print(x.is_pointer_to_remote)
print(x.id)

False
9324432963


In [23]:
y


 2  2
 2  2
[torch.FloatTensor of size 2x2]

In [24]:
x.fill_(0)


 0  0
 0  0
[torch.FloatTensor of size 2x2]

In [25]:
print(x)


 0  0
 0  0
[torch.FloatTensor of size 2x2]



Case when tensor isn't local

In [26]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [27]:
x.normal_()

Placeholder print for sending command to worker A1
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],), {})

In [28]:
x.uniform_()

Placeholder print for sending command to worker A1
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],), {})

In [29]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],
  [ torch.FloatTensor - Location:other_guy ]),
 {})

In [30]:
try:
    torch.add(x,y) # This should throw an error, since their attributes say they're not on the same machine.
except NotImplementedError:
    print('booped!')

booped!


<a id='double_test'></a>

### DoubleTensor

In [31]:
y = torch.DoubleTensor([[2,2],[2,2]])
z = torch.DoubleTensor(([[1,1],[1,1]]))

In [32]:
x = y.add(z)

In [33]:
print(x.is_pointer_to_remote)
print(x.id)

False
5171645209


In [34]:
x


 3  3
 3  3
[torch.DoubleTensor of size 2x2]

In [35]:
x.fill_(0)


 0  0
 0  0
[torch.DoubleTensor of size 2x2]

In [36]:
print(x)


 0  0
 0  0
[torch.DoubleTensor of size 2x2]



Case when tensor isn't local

In [37]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [38]:
x.normal_()

Placeholder print for sending command to worker A1
normal_
[<class 'torch.DoubleTensor'>]
[]

Placeholder print for sending command to worker B1
normal_
[<class 'torch.DoubleTensor'>]
[]

Placeholder print for sending command to worker B2
normal_
[<class 'torch.DoubleTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.DoubleTensor'> - Location:other_guy ],), {})

In [39]:
x.uniform_()

Placeholder print for sending command to worker A1
uniform_
[<class 'torch.DoubleTensor'>]
[]

Placeholder print for sending command to worker B1
uniform_
[<class 'torch.DoubleTensor'>]
[]

Placeholder print for sending command to worker B2
uniform_
[<class 'torch.DoubleTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.DoubleTensor'> - Location:other_guy ],), {})

In [40]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.DoubleTensor'>, <class 'torch.DoubleTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.DoubleTensor'>, <class 'torch.DoubleTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.DoubleTensor'>, <class 'torch.DoubleTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.DoubleTensor'> - Location:other_guy ],
  [ <class 'torch.DoubleTensor'> - Location:other_guy ]),
 {})

<a id='half_test'></a>

### HalfTensor
This one's weird, I think it might be underdeveloped on their end.

In [41]:
y = torch.HalfTensor([[2,2],[2,2]])
z = torch.HalfTensor(([[1,1],[1,1]]))

In [42]:
y.float()


 2  2
 2  2
[torch.FloatTensor of size 2x2]

Case when tensor isn't local

In [43]:
y.is_pointer_to_remote = True
y.owner = 'other_guy'

In [44]:
y.float()

Placeholder print for sending command to worker A1
float
[<class 'torch.HalfTensor'>]
[]

Placeholder print for sending command to worker B1
float
[<class 'torch.HalfTensor'>]
[]

Placeholder print for sending command to worker B2
float
[<class 'torch.HalfTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.HalfTensor'> - Location:other_guy ],), {})

In [45]:
torch.add(y, y)

Placeholder print for sending command to worker A1
add
[<class 'torch.HalfTensor'>, <class 'torch.HalfTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.HalfTensor'>, <class 'torch.HalfTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.HalfTensor'>, <class 'torch.HalfTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.HalfTensor'> - Location:other_guy ],
  [ <class 'torch.HalfTensor'> - Location:other_guy ]),
 {})

In [46]:
a = torch.HalfTensor([[2,2],[2,2]])
b = torch.HalfTensor(([[1,1],[1,1]]))

In [47]:
try:
    torch.add(a,b)
except:
    print('HalfTensor is weird')

HalfTensor is weird


<a id='byte_test'></a>

### ByteTensor

In [48]:
y = torch.ByteTensor([[0,1],[1,0]])
z = torch.ByteTensor(([[0,0],[1,1]]))

In [49]:
z


 0  0
 1  1
[torch.ByteTensor of size 2x2]

In [50]:
x = y.add(z)

In [51]:
y.fill_(255)


 255  255
 255  255
[torch.ByteTensor of size 2x2]

In [52]:
print(x.is_pointer_to_remote)
print(x.id)

False
6447598411


In [53]:
x


 0  1
 2  1
[torch.ByteTensor of size 2x2]

In [54]:
x.t()


 0  2
 1  1
[torch.ByteTensor of size 2x2]

In [55]:
y.add_(1)


 0  0
 0  0
[torch.ByteTensor of size 2x2]

In [56]:
print(x)
print(y)


 0  1
 2  1
[torch.ByteTensor of size 2x2]


 0  0
 0  0
[torch.ByteTensor of size 2x2]



In [57]:
x.t_()
print(x)


 0  2
 1  1
[torch.ByteTensor of size 2x2]



Case when tensor isn't local

In [58]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [59]:
#x.normal_()

In [60]:
#x.uniform_()

In [61]:
x.t()

Placeholder print for sending command to worker A1
t
[<class 'torch.ByteTensor'>]
[]

Placeholder print for sending command to worker B1
t
[<class 'torch.ByteTensor'>]
[]

Placeholder print for sending command to worker B2
t
[<class 'torch.ByteTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.ByteTensor'> - Location:other_guy ],), {})

In [62]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.ByteTensor'>, <class 'torch.ByteTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.ByteTensor'>, <class 'torch.ByteTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.ByteTensor'>, <class 'torch.ByteTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.ByteTensor'> - Location:other_guy ],
  [ <class 'torch.ByteTensor'> - Location:other_guy ]),
 {})

<a id='char_test'></a>

### CharTensor

In [63]:
y = torch.CharTensor([[0,1],[1,0]])
z = torch.CharTensor(([[0,0],[1,1]]))

In [64]:
z


 0  0
 1  1
[torch.CharTensor of size 2x2]

In [65]:
x = y.add(z)

In [66]:
y.fill_(255)


-1 -1
-1 -1
[torch.CharTensor of size 2x2]

In [67]:
print(x.is_pointer_to_remote)
print(x.id)

False
3428024029


In [68]:
x


 0  1
 2  1
[torch.CharTensor of size 2x2]

In [69]:
x.t()


 0  2
 1  1
[torch.CharTensor of size 2x2]

In [70]:
y.add_(1)


 0  0
 0  0
[torch.CharTensor of size 2x2]

In [71]:
print(x)
print(y)


 0  1
 2  1
[torch.CharTensor of size 2x2]


 0  0
 0  0
[torch.CharTensor of size 2x2]



In [72]:
x.t_()
print(x)


 0  2
 1  1
[torch.CharTensor of size 2x2]



Case when tensor isn't local

In [73]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [74]:
#x.normal_()

In [75]:
#x.uniform_()

In [76]:
x.t()

Placeholder print for sending command to worker A1
t
[<class 'torch.CharTensor'>]
[]

Placeholder print for sending command to worker B1
t
[<class 'torch.CharTensor'>]
[]

Placeholder print for sending command to worker B2
t
[<class 'torch.CharTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.CharTensor'> - Location:other_guy ],), {})

In [77]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.CharTensor'>, <class 'torch.CharTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.CharTensor'>, <class 'torch.CharTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.CharTensor'>, <class 'torch.CharTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.CharTensor'> - Location:other_guy ],
  [ <class 'torch.CharTensor'> - Location:other_guy ]),
 {})

<a id='short_test'></a>

### ShortTensor

In [78]:
y = torch.ShortTensor([[1,2],[3,4]])
z = torch.ShortTensor(([[1,1],[1,1]]))

In [79]:
x = y.add(z)

In [80]:
print(x.is_pointer_to_remote)
print(x.id)

False
11527967


In [81]:
x


 2  3
 4  5
[torch.ShortTensor of size 2x2]

In [82]:
x.t()


 2  4
 3  5
[torch.ShortTensor of size 2x2]

In [83]:
y.fill_(0)


 0  0
 0  0
[torch.ShortTensor of size 2x2]

In [84]:
print(x)
print(y)


 2  3
 4  5
[torch.ShortTensor of size 2x2]


 0  0
 0  0
[torch.ShortTensor of size 2x2]



In [85]:
x.t_()
print(x)


 2  4
 3  5
[torch.ShortTensor of size 2x2]



Case when tensor isn't local

In [86]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [87]:
#x.normal_()

In [88]:
#x.uniform_()

In [89]:
x.t()

Placeholder print for sending command to worker A1
t
[<class 'torch.ShortTensor'>]
[]

Placeholder print for sending command to worker B1
t
[<class 'torch.ShortTensor'>]
[]

Placeholder print for sending command to worker B2
t
[<class 'torch.ShortTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.ShortTensor'> - Location:other_guy ],), {})

In [90]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.ShortTensor'>, <class 'torch.ShortTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.ShortTensor'>, <class 'torch.ShortTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.ShortTensor'>, <class 'torch.ShortTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.ShortTensor'> - Location:other_guy ],
  [ <class 'torch.ShortTensor'> - Location:other_guy ]),
 {})

<a id='int_test'></a>

### IntTensor

In [91]:
y = torch.IntTensor([[1,2],[3,4]])
z = torch.IntTensor(([[1,1],[1,1]]))

In [92]:
x = y.add(z)

In [93]:
print(x.is_pointer_to_remote)
print(x.id)

False
8865562600


In [94]:
x


 2  3
 4  5
[torch.IntTensor of size 2x2]

In [95]:
x.t()


 2  4
 3  5
[torch.IntTensor of size 2x2]

In [96]:
y.fill_(0)


 0  0
 0  0
[torch.IntTensor of size 2x2]

In [97]:
print(x)
print(y)


 2  3
 4  5
[torch.IntTensor of size 2x2]


 0  0
 0  0
[torch.IntTensor of size 2x2]



In [98]:
x.t_()
print(x)


 2  4
 3  5
[torch.IntTensor of size 2x2]



Case when tensor isn't local

In [99]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [100]:
#x.normal_()

In [101]:
#x.uniform_()

In [102]:
x.t()

Placeholder print for sending command to worker A1
t
[<class 'torch.IntTensor'>]
[]

Placeholder print for sending command to worker B1
t
[<class 'torch.IntTensor'>]
[]

Placeholder print for sending command to worker B2
t
[<class 'torch.IntTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.IntTensor'> - Location:other_guy ],), {})

In [103]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.IntTensor'>, <class 'torch.IntTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.IntTensor'>, <class 'torch.IntTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.IntTensor'>, <class 'torch.IntTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.IntTensor'> - Location:other_guy ],
  [ <class 'torch.IntTensor'> - Location:other_guy ]),
 {})

<a id='long_test'></a>

### LongTensor

In [104]:
y = torch.LongTensor([[1,2],[3,4]])
z = torch.LongTensor(([[1,1],[1,1]]))

In [105]:
x = y.add(z)

In [106]:
x.owner

'QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc'

In [107]:
print(x.is_pointer_to_remote)
print(x.id)

False
5264969863


In [108]:
x.t()


 2  4
 3  5
[torch.LongTensor of size 2x2]

In [109]:
x.t_()


 2  4
 3  5
[torch.LongTensor of size 2x2]

In [110]:
torch.add(x, x)


  4   8
  6  10
[torch.LongTensor of size 2x2]

Case when tensor isn't local

In [111]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [112]:
#x.normal_()

In [113]:
#x.uniform_()

In [114]:
x.t()

Placeholder print for sending command to worker A1
t
[<class 'torch.LongTensor'>]
[]

Placeholder print for sending command to worker B1
t
[<class 'torch.LongTensor'>]
[]

Placeholder print for sending command to worker B2
t
[<class 'torch.LongTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.LongTensor'> - Location:other_guy ],), {})

In [115]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.LongTensor'>, <class 'torch.LongTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.LongTensor'>, <class 'torch.LongTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.LongTensor'>, <class 'torch.LongTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ <class 'torch.LongTensor'> - Location:other_guy ],
  [ <class 'torch.LongTensor'> - Location:other_guy ]),
 {})

<a id='var_test'></a>

### TODO: Variable [(back to hook)](#hook)

In [116]:
y = Variable(torch.FloatTensor([[1,2],[3,4]]), requires_grad = True)
z = Variable(torch.FloatTensor(([[1,1],[1,1]])), requires_grad = True)

In [117]:
y.data_registered

False

In [118]:
z.grad_registered

False

In [119]:
y.grad

In [120]:
y.owner

'QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc'

In [121]:
y.data.owner

'QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc'

In [122]:
x = y + z

In [123]:
x.owner

'QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc'

In [124]:
x.requires_grad

True

In [125]:
y.id

384345516

In [126]:
y.data


 1  2
 3  4
[torch.FloatTensor of size 2x2]

In [127]:
service_self.register_object(x.data, False).id

5976705432

In [128]:
x.data.owner

'QmXJMbiCqQdFCUjwy63GMUDDKCfEabJRYo2RHPjheCW8mc'

In [129]:
x = y.add(z)

In [130]:
x.data


 2  3
 4  5
[torch.FloatTensor of size 2x2]

In [131]:
x.data_registered

True

In [132]:
y.data_registered

True

In [133]:
x.sum().backward()

In [134]:
x.data_registered

True

In [135]:
x.data


 2  3
 4  5
[torch.FloatTensor of size 2x2]

TODO: Make sure data_registered and grad_registered are set to False on leaf variable grads (likely through force_hook above)

In [137]:
y.grad.id

5987102910

In [ ]:
y.grad.id